In [ ]:
# %% [markdown]
# # ATG Debate System (Colab Version)
#
# **Two-Agent Debate Simulation with LangGraph**
#
# This notebook implements:
# - Scientist vs Philosopher debate
# - 8 rounds of structured arguments
# - Memory and state management
# - Automated judging
#
# *Requires Anthropic API key for Claude models*

# %% [code]
# @title Step 1: Install Required Packages
!pip install -q langgraph langchain python-dotenv
!pip install -q pydantic==2.6.4  # Required for Colab compatibility

# %% [code]
# @title Step 2: Enter API Key
import os
from getpass import getpass

api_key = getpass("Enter your Anthropic API key: ")
os.environ["ANTHROPIC_API_KEY"] = api_key

# %% [code]
# @title Step 3: Define Core Components
from langgraph.graph import Graph, END
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage
from typing import Dict, List, TypedDict
import logging

# Define state structure
class DebateState(TypedDict):
    topic: str
    round: int
    history: List
    current_speaker: str
    current_argument: str

# Initialize logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('debate_log.txt'),
        logging.StreamHandler()
    ]
)

# %% [code]
# @title Step 4: Create Debate Nodes
# User Input Node
def user_input_node(state: DebateState):
    topic = input("Enter debate topic: ")
    return {"topic": topic, "round": 0, "history": []}

# Scientist Agent
scientist_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a logical scientist. Present evidence-based arguments about risks and empirical data."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "Debate topic: {topic}\nRound {round}: Present your scientific argument"),
])

scientist_chain = scientist_prompt | ChatAnthropic(model="claude-3-haiku-20240307")

def scientist_node(state: DebateState):
    response = scientist_chain.invoke({
        "topic": state["topic"],
        "round": state["round"],
        "history": state["history"]
    })

    logging.info(f"[Round {state['round']}] Scientist: {response.content}")
    return {
        "history": state["history"] + [{"role": "scientist", "content": response.content}],
        "current_speaker": "scientist",
        "current_argument": response.content
    }

# Philosopher Agent
philosopher_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a philosopher. Argue about ethics, morality, and societal implications."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "Debate topic: {topic}\nRound {round}: Present your philosophical argument"),
])

philosopher_chain = philosopher_prompt | ChatAnthropic(model="claude-3-haiku-20240307")

def philosopher_node(state: DebateState):
    response = philosopher_chain.invoke({
        "topic": state["topic"],
        "round": state["round"],
        "history": state["history"]
    })

    logging.info(f"[Round {state['round']}] Philosopher: {response.content}")
    return {
        "history": state["history"] + [{"role": "philosopher", "content": response.content}],
        "current_speaker": "philosopher",
        "current_argument": response.content
    }

# Judge Node
judge_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert debate judge. Analyze this debate and:
    1. Summarize key points from both sides
    2. Evaluate argument strength
    3. Declare a winner with specific reasons"""),
    ("human", """Debate Topic: {topic}

    Full Debate History:
    {history}

    Please provide your verdict:""")
])

judge_chain = judge_prompt | ChatAnthropic(model="claude-3-opus-20240229")

def judge_node(state: DebateState):
    # Format history for judge
    debate_transcript = "\n".join(
        f"[{msg['role'].title()}] {msg['content']}"
        for msg in state["history"]
    )

    response = judge_chain.invoke({
        "topic": state["topic"],
        "history": debate_transcript
    })

    logging.info(f"\n[JUDGE VERDICT]\n{response.content}")
    return {"verdict": response.content}

# %% [code]
# @title Step 5: Build the Workflow
workflow = Graph()

# Add nodes
workflow.add_node("user_input", user_input_node)
workflow.add_node("scientist", scientist_node)
workflow.add_node("philosopher", philosopher_node)
workflow.add_node("judge", judge_node)

# Set up edges
workflow.set_entry_point("user_input")

# Define routing logic
def route_rounds(state: DebateState):
    if state["round"] >= 8:
        return "judge"
    elif state["round"] % 2 == 1:  # Odd rounds: scientist
        return "scientist"
    else:  # Even rounds: philosopher
        return "philosopher"

workflow.add_conditional_edges(
    "user_input",
    lambda x: "scientist",  # First round always scientist
)

workflow.add_conditional_edges(
    "scientist",
    route_rounds
)

workflow.add_conditional_edges(
    "philosopher",
    route_rounds
)

workflow.add_edge("judge", END)

# Compile
app = workflow.compile()

# %% [code]
# @title Step 6: Run the Debate
print("\n=== ATG Debate System ===")
print("Starting debate...\n")

# Initialize state
state = {"round": 0}

# Get user input
state = user_input_node(state)

# Run 8 rounds alternating between agents
for round_num in range(1, 9):
    state["round"] = round_num

    if round_num % 2 == 1:
        state = scientist_node(state)
    else:
        state = philosopher_node(state)

# Final judgment
verdict = judge_node(state)

print("\n=== Debate Concluded ===")
print(verdict["verdict"])

# %% [code]
# @title Step 7: View Logs
print("\nLog file contents:")
!cat debate_log.txt

# %% [markdown]
# **How to Use:**
# 1. Run all cells (Runtime > Run all)
# 2. When prompted, enter your Anthropic API key
# 3. Enter debate topic when asked
# 4. Watch the debate unfold in the output
# 5. Final verdict will appear at the end
#
# **Notes:**
# - Uses Claude Haiku for agents (fast/affordable)
# - Uses Claude Opus for judging (higher quality)
# - Full log saved to `debate_log.txt`
# - Change models in the agent definitions if needed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.5.0 requires pydantic>=2.7.4, but you have pydantic 2.6.4 which is incompatible.
albumentations 2.0.8 requires pydantic>=2.9.2, but you have pydantic 2.6.4 which is incompatible.
